## 7.4

Return to the permeability problem outlined in Exercise 6.2. Train several nonlinear regression models and evaluate the resampling and test set performance.

**(a)** Which nonlinear regression model gives the optimal resampling and test set performance?

**(b)** Do any of the nonlinear models outperform the optimal linear model you previously developed in Exercise 6.2? If so, what might this tell you about the underlying relationship between the predictors and the response?

**(c)** Would you recommend any of the models you have developed to replace the permeability laboratory experiment?

In [9]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from matplotlib import pyplot as plt
from matplotlib import cm
from pyearth import Earth
import tensorflow as tf

In [2]:
from sklearn.model_selection import train_test_split

def split_data(df, target, val_size=0.2, test_size=0.2, drop_columns=[]):
    x={}
    y={}
    
    train_val, test = train_test_split(df, test_size=test_size)
    train, val = train_test_split(train_val, test_size=val_size/(1-test_size))
    ds = {}
    x["train"] = train.drop(columns=[target]+drop_columns)
    x["val"] = val.drop(columns=[target]+drop_columns)
    x["train_val"] = train_val.drop(columns=[target]+drop_columns)
    x["test"] = test.drop(columns=[target]+drop_columns)
    y["train"] = train[target]
    y["val"] = val[target]
    y["train_val"] = train_val[target]
    y["test"] = test[target]
    
    return x,y

In [39]:
fingerprints = pd.read_csv("data/fingerprints.csv", index_col="ID", header=0)
permeability = pd.read_csv("data/permeability.csv", index_col="ID", header=0)

data = pd.concat([fingerprints, permeability], axis="columns")
x, y = split_data(data, "permeability")

In [ ]:
# Non-linear regression models
# Neural Networks
# - Implement a single example of nn regressor
# - Implement cross validation for nn
# - Test cross validation for baseline architecture
# - Google and implement better architectures.
# - Preprocessing

# MARS
# SVM
# KNN

In [40]:
x_tensor, y_tensor = {}, {}

for key in x:
    x_tensor[key] = tf.convert_to_tensor(x[key])
    y_tensor[key] = tf.convert_to_tensor(y[key])

In [102]:
model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [103]:
predictions = model(tf.convert_to_tensor(x["train"].iloc[0:1]))

In [104]:
loss_fn = tf.keras.losses.MeanSquaredError()

In [105]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['mse'])

In [106]:
model.fit(x_tensor["train_val"], y_tensor["train_val"], validation_split=0.2, epochs=500)

Epoch 1/500
4/4 [==============================] - 1s 75ms/step - loss: 368.6597 - mse: 368.6597 - val_loss: 391.0689 - val_mse: 391.0689
Epoch 2/500
4/4 [==============================] - 0s 25ms/step - loss: 362.4326 - mse: 362.4326 - val_loss: 379.7231 - val_mse: 379.7231
Epoch 3/500
4/4 [==============================] - 0s 25ms/step - loss: 344.3876 - mse: 344.3876 - val_loss: 369.0631 - val_mse: 369.0631
Epoch 4/500
4/4 [==============================] - 0s 26ms/step - loss: 349.7387 - mse: 349.7387 - val_loss: 359.2155 - val_mse: 359.2155
Epoch 5/500
4/4 [==============================] - 0s 27ms/step - loss: 297.7382 - mse: 297.7382 - val_loss: 350.5259 - val_mse: 350.5259
Epoch 6/500
4/4 [==============================] - 0s 29ms/step - loss: 301.5932 - mse: 301.5932 - val_loss: 342.7527 - val_mse: 342.7527
Epoch 7/500
4/4 [==============================] - 0s 28ms/step - loss: 314.0630 - mse: 314.0630 - val_loss: 335.6975 - val_mse: 335.6975
Epoch 8/500
4/4 [=================

4/4 [==============================] - 0s 26ms/step - loss: 201.4146 - mse: 201.4146 - val_loss: 230.4446 - val_mse: 230.4446
Epoch 61/500
4/4 [==============================] - 0s 23ms/step - loss: 208.3600 - mse: 208.3600 - val_loss: 229.2972 - val_mse: 229.2972
Epoch 62/500
4/4 [==============================] - 0s 28ms/step - loss: 160.5960 - mse: 160.5960 - val_loss: 228.1027 - val_mse: 228.1027
Epoch 63/500
4/4 [==============================] - 0s 24ms/step - loss: 177.1333 - mse: 177.1333 - val_loss: 227.0445 - val_mse: 227.0445
Epoch 64/500
4/4 [==============================] - 0s 22ms/step - loss: 196.8527 - mse: 196.8527 - val_loss: 225.9646 - val_mse: 225.9646
Epoch 65/500
4/4 [==============================] - 0s 23ms/step - loss: 180.0250 - mse: 180.0250 - val_loss: 224.9684 - val_mse: 224.9684
Epoch 66/500
4/4 [==============================] - 0s 25ms/step - loss: 180.2547 - mse: 180.2547 - val_loss: 224.0053 - val_mse: 224.0053
Epoch 67/500
4/4 [======================

Epoch 119/500
4/4 [==============================] - 0s 26ms/step - loss: 127.0470 - mse: 127.0470 - val_loss: 185.8296 - val_mse: 185.8296
Epoch 120/500
4/4 [==============================] - 0s 24ms/step - loss: 141.1483 - mse: 141.1483 - val_loss: 185.3732 - val_mse: 185.3732
Epoch 121/500
4/4 [==============================] - 0s 33ms/step - loss: 138.6674 - mse: 138.6674 - val_loss: 185.0213 - val_mse: 185.0213
Epoch 122/500
4/4 [==============================] - 0s 29ms/step - loss: 126.3961 - mse: 126.3961 - val_loss: 184.7365 - val_mse: 184.7365
Epoch 123/500
4/4 [==============================] - 0s 27ms/step - loss: 144.8398 - mse: 144.8398 - val_loss: 184.4267 - val_mse: 184.4267
Epoch 124/500
4/4 [==============================] - 0s 26ms/step - loss: 161.4555 - mse: 161.4555 - val_loss: 184.0392 - val_mse: 184.0392
Epoch 125/500
4/4 [==============================] - 0s 26ms/step - loss: 134.8119 - mse: 134.8119 - val_loss: 183.7393 - val_mse: 183.7393
Epoch 126/500
4/4 [=

4/4 [==============================] - 0s 28ms/step - loss: 107.7509 - mse: 107.7509 - val_loss: 167.1527 - val_mse: 167.1527
Epoch 178/500
4/4 [==============================] - 0s 28ms/step - loss: 91.8409 - mse: 91.8409 - val_loss: 166.8894 - val_mse: 166.8894
Epoch 179/500
4/4 [==============================] - 0s 24ms/step - loss: 117.5322 - mse: 117.5322 - val_loss: 166.7604 - val_mse: 166.7604
Epoch 180/500
4/4 [==============================] - 0s 25ms/step - loss: 111.1537 - mse: 111.1537 - val_loss: 166.6372 - val_mse: 166.6372
Epoch 181/500
4/4 [==============================] - 0s 28ms/step - loss: 95.4249 - mse: 95.4249 - val_loss: 166.5011 - val_mse: 166.5011
Epoch 182/500
4/4 [==============================] - 0s 28ms/step - loss: 84.4520 - mse: 84.4520 - val_loss: 166.3521 - val_mse: 166.3521
Epoch 183/500
4/4 [==============================] - 0s 25ms/step - loss: 117.0416 - mse: 117.0416 - val_loss: 166.1445 - val_mse: 166.1445
Epoch 184/500
4/4 [=====================

Epoch 236/500
4/4 [==============================] - 0s 28ms/step - loss: 102.3563 - mse: 102.3563 - val_loss: 155.4947 - val_mse: 155.4947
Epoch 237/500
4/4 [==============================] - 0s 27ms/step - loss: 90.8056 - mse: 90.8056 - val_loss: 155.3484 - val_mse: 155.3484
Epoch 238/500
4/4 [==============================] - 0s 31ms/step - loss: 94.2112 - mse: 94.2112 - val_loss: 155.3562 - val_mse: 155.3562
Epoch 239/500
4/4 [==============================] - 0s 31ms/step - loss: 106.7627 - mse: 106.7627 - val_loss: 155.3942 - val_mse: 155.3942
Epoch 240/500
4/4 [==============================] - 0s 37ms/step - loss: 86.5408 - mse: 86.5408 - val_loss: 155.3512 - val_mse: 155.3512
Epoch 241/500
4/4 [==============================] - 0s 36ms/step - loss: 84.6800 - mse: 84.6800 - val_loss: 155.3392 - val_mse: 155.3392
Epoch 242/500
4/4 [==============================] - 0s 34ms/step - loss: 92.5621 - mse: 92.5621 - val_loss: 155.1716 - val_mse: 155.1716
Epoch 243/500
4/4 [===========

4/4 [==============================] - 0s 27ms/step - loss: 81.4280 - mse: 81.4280 - val_loss: 150.1367 - val_mse: 150.1367
Epoch 296/500
4/4 [==============================] - 0s 29ms/step - loss: 77.5754 - mse: 77.5754 - val_loss: 149.7535 - val_mse: 149.7535
Epoch 297/500
4/4 [==============================] - 0s 27ms/step - loss: 91.1606 - mse: 91.1606 - val_loss: 149.4459 - val_mse: 149.4459
Epoch 298/500
4/4 [==============================] - 0s 25ms/step - loss: 83.9179 - mse: 83.9179 - val_loss: 149.2715 - val_mse: 149.2715
Epoch 299/500
4/4 [==============================] - 0s 26ms/step - loss: 95.3735 - mse: 95.3735 - val_loss: 149.1657 - val_mse: 149.1657
Epoch 300/500
4/4 [==============================] - 0s 28ms/step - loss: 75.8717 - mse: 75.8717 - val_loss: 149.0138 - val_mse: 149.0138
Epoch 301/500
4/4 [==============================] - 0s 28ms/step - loss: 84.0228 - mse: 84.0228 - val_loss: 148.7842 - val_mse: 148.7842
Epoch 302/500
4/4 [=============================

4/4 [==============================] - 0s 28ms/step - loss: 85.6112 - mse: 85.6112 - val_loss: 146.0188 - val_mse: 146.0188
Epoch 355/500
4/4 [==============================] - 0s 29ms/step - loss: 77.8904 - mse: 77.8904 - val_loss: 146.1100 - val_mse: 146.1100
Epoch 356/500
4/4 [==============================] - 0s 27ms/step - loss: 81.0103 - mse: 81.0103 - val_loss: 146.1871 - val_mse: 146.1871
Epoch 357/500
4/4 [==============================] - 0s 26ms/step - loss: 76.1320 - mse: 76.1320 - val_loss: 146.3007 - val_mse: 146.3007
Epoch 358/500
4/4 [==============================] - 0s 25ms/step - loss: 83.3917 - mse: 83.3917 - val_loss: 146.0586 - val_mse: 146.0586
Epoch 359/500
4/4 [==============================] - 0s 28ms/step - loss: 76.1064 - mse: 76.1064 - val_loss: 146.0341 - val_mse: 146.0341
Epoch 360/500
4/4 [==============================] - 0s 30ms/step - loss: 82.8594 - mse: 82.8594 - val_loss: 146.1516 - val_mse: 146.1516
Epoch 361/500
4/4 [=============================

4/4 [==============================] - 0s 28ms/step - loss: 65.1932 - mse: 65.1932 - val_loss: 143.5510 - val_mse: 143.5510
Epoch 414/500
4/4 [==============================] - 0s 25ms/step - loss: 73.1216 - mse: 73.1216 - val_loss: 143.5290 - val_mse: 143.5290
Epoch 415/500
4/4 [==============================] - 0s 28ms/step - loss: 62.0915 - mse: 62.0915 - val_loss: 143.5366 - val_mse: 143.5366
Epoch 416/500
4/4 [==============================] - 0s 27ms/step - loss: 64.4270 - mse: 64.4270 - val_loss: 143.2899 - val_mse: 143.2899
Epoch 417/500
4/4 [==============================] - 0s 29ms/step - loss: 74.8887 - mse: 74.8887 - val_loss: 142.9008 - val_mse: 142.9008
Epoch 418/500
4/4 [==============================] - 0s 26ms/step - loss: 67.0922 - mse: 67.0922 - val_loss: 142.6711 - val_mse: 142.6711
Epoch 419/500
4/4 [==============================] - 0s 26ms/step - loss: 87.2087 - mse: 87.2087 - val_loss: 142.2297 - val_mse: 142.2297
Epoch 420/500
4/4 [=============================

4/4 [==============================] - 0s 25ms/step - loss: 75.8249 - mse: 75.8249 - val_loss: 140.6451 - val_mse: 140.6451
Epoch 473/500
4/4 [==============================] - 0s 26ms/step - loss: 65.9421 - mse: 65.9421 - val_loss: 140.5981 - val_mse: 140.5981
Epoch 474/500
4/4 [==============================] - 0s 30ms/step - loss: 74.9956 - mse: 74.9956 - val_loss: 140.5633 - val_mse: 140.5633
Epoch 475/500
4/4 [==============================] - 0s 31ms/step - loss: 71.7954 - mse: 71.7954 - val_loss: 140.4475 - val_mse: 140.4475
Epoch 476/500
4/4 [==============================] - 0s 25ms/step - loss: 55.8500 - mse: 55.8500 - val_loss: 140.6779 - val_mse: 140.6779
Epoch 477/500
4/4 [==============================] - 0s 26ms/step - loss: 67.4376 - mse: 67.4376 - val_loss: 140.9174 - val_mse: 140.9174
Epoch 478/500
4/4 [==============================] - 0s 25ms/step - loss: 69.7139 - mse: 69.7139 - val_loss: 140.9778 - val_mse: 140.9778
Epoch 479/500
4/4 [=============================

In [67]:
loss_fn(model.predict(x_tensor["train"]), y_tensor["train"])

<tf.Tensor: shape=(), dtype=float64, numpy=483.5649108886719>

In [73]:
for i in range(10):
    print(model.predict(x_tensor["val"])[i], y_tensor["val"][i].numpy())


[-3.302729] 0.725
[9.871213] 0.52
[8.774123] 18.915
[49.07919] 48.51
[5.311449] 5.75
[7.9114447] 8.18
[2.6542113] 2.59
[4.348044] 5.275
[0.65121925] 1.195
[11.357814] 12.49


In [76]:
(model.predict(x_tensor["test"]) - y_tensor["test"])**2

<tf.Tensor: shape=(33, 33), dtype=float64, numpy=
array([[1.79636899e+01, 1.10001053e+01, 9.24166777e+01, ...,
        1.43765389e+01, 2.33613625e+01, 8.52443826e+02],
       [4.52924805e-01, 4.73619669e+01, 3.65782663e+01, ...,
        5.41254949e+01, 1.60781610e+00, 1.07334885e+03],
       [2.85608751e+01, 1.66390330e+02, 9.46327639e-04, ...,
        1.78870230e+02, 2.25552574e+01, 1.50382927e+03],
       ...,
       [5.47602749e+01, 2.40192413e-02, 1.63201100e+02, ...,
        3.96876593e-01, 6.39203221e+01, 6.77820258e+02],
       [1.92464025e+01, 1.42613088e+02, 9.76001336e-01, ...,
        1.54183682e+02, 1.43798116e+01, 1.43050915e+03],
       [2.82277623e+02, 8.54907293e+01, 4.91780285e+02, ...,
        7.69325406e+01, 3.02624980e+02, 2.76685974e+02]])>

In [101]:
print(model.predict(x_tensor["train"]).reshape(-1) - y_tensor["train"].numpy())

[-5.61848565e-01 -2.33072815e-01 -3.07776890e-01 -2.02120514e-01
  5.59483959e+00 -2.18309784e-02 -4.47429938e-01 -1.66054828e+01
 -6.87191391e-02 -1.94617209e-01 -5.81196895e-01 -9.85219707e-01
 -2.07850795e-01 -3.15737901e-01 -5.08495445e+00 -1.07811127e+00
  1.76788330e-03  3.74531708e-01 -8.89389496e-01  4.45407867e-01
 -1.03707886e-02 -9.78854663e-01 -5.15252056e-01  3.67117262e-01
  1.98477917e-01 -6.69370899e-01  6.28100891e-01  5.69189301e-01
 -6.18096771e-01 -1.31011391e-01 -3.42424073e-01 -2.42300663e-01
 -1.33263535e+00  8.94380951e-02  8.39080133e-01  4.86600571e-01
 -9.00277328e-02 -5.43969498e-01  2.75968475e-01  1.94514465e-02
 -7.49956323e+00  7.13500323e-01 -3.12852605e+00 -4.84027519e-01
 -1.76259861e-01  5.69698349e+00  2.76202927e-01 -6.28615065e-01
 -1.61904092e-01 -4.85511475e-01 -7.37370800e+00  8.37381825e-01
 -8.97504516e-01 -5.20250506e-01 -4.38496723e-01  3.01050892e-01
  1.41518421e-01  4.18551493e-01 -2.51279311e-01 -2.15089612e-01
 -4.85414963e-01 -6.52137

In [90]:
.shape

(33,)

In [86]:
y_tensor["test"].numpy().shape

(33,)